# Quality Control

This script will run a quality control pipeline in the genotype files.

## Preliminaries

First, let's import modules and set paths

In [1]:
import glob, os, shutil, subprocess
import pandas as pd
from GenotypeQC import QC_procedure

In [2]:
projpath  = os.path.realpath("..")
pathgenos = os.path.join(projpath, "DataBases", "Genotypes")
pathdat   = os.path.join(projpath, "DataBases")
outpath   = os.path.join(pathgenos, "02_Clean")

# Retain common IDs

For each dataset, we will retain only the common IDs.
First, we will need to extract the FID for each IID, from the fam files

In [3]:
#Reading common IDS
os.chdir(pathdat)
common_ids = pd.read_csv("common_ids.txt", header=None, names = ["ID"])

#Creating empty dataframe
common_FID_ID = pd.DataFrame(columns=["FID", "ID"])

#Reading fam files and adding to empty dataframe
os.chdir(os.path.join(pathgenos, "01_Original"))
for fam in glob.glob("*.fam"):
    #UIUC2014 are repeated
    if fam == "UIUC2014_168ppl_703K_hg19_ATGC.fam":
        pass
    else:
        famfile = pd.read_csv(fam, header=None, sep=" ", names=["FID", "ID"], usecols=["FID", "ID"], dtype=object )
        inter = pd.merge(common_ids, famfile, on="ID")[ ["FID", "ID"]]
        common_FID_ID = pd.concat([common_FID_ID, inter], ignore_index=True)

Now, we will save the final dataset

In [4]:
os.chdir(pathdat)
common_FID_ID.to_csv("common_fid_id.txt", sep=" ", header=False, index=False)

Finally, we will extract for each dataset only the common IDs, in map ped file for the lift over process

In [9]:
os.chdir(os.path.join(pathgenos, "01_Original"))
pathsave_commonid = os.path.join(pathgenos, "01_Original", "Extract_Common_ID")
keepfile = os.path.join(pathdat, "common_fid_id.txt")

for bedfile in glob.glob("*.bed"):
    filename = bedfile.split(".")[0]
    outfile  = filename + "_CID"
    if "CV_" in filename or "Euro180_" in filename:
        subprocess.run(["plink", "--bfile", filename, "--keep", keepfile, "--recode", "--out", os.path.join(pathsave_commonid, outfile)])
    else:
        subprocess.run(["plink", "--bfile", filename, "--keep", keepfile, "--make-bed", "--out", os.path.join(pathsave_commonid, outfile)])

# Lift Over

Before the QC procedure, we will make sure to lift over the datasets from hg18 to hg19. We will use [LiftOverPlink](https://github.com/sritchie73/liftOverPlink) as a wraper for [liftOver](https://genome.sph.umich.edu/wiki/LiftOver). We will also [download](http://hgdownload.cse.ucsc.edu/goldenPath/hg18/liftOver/hg18ToHg19.over.chain.gz) the chain file that tells how to lift over from hg18 to hg19. This is only done for the CV, and Euro datasets

In [10]:
os.chdir(os.path.join(pathgenos, "01_Original", "Extract_Common_ID"))
liftoverplink = os.path.join(projpath, "Code", "liftOverPlink")
badlifts      = os.path.join(projpath, "Code", "rmBadLifts")
chainfile     = os.path.join(pathdat, "hg18ToHg19.over.chain.gz")
for mapfile in glob.glob("*.map"):
    if "CV_" in mapfile or "Euro180_" in mapfile:
        outlifted    = mapfile.split(".")[0] + "_lifted"
        loglifted    = mapfile.split(".")[0] + "_bad_lifted.dat"
        
        %run $liftoverplink --map $mapfile --out lifted --chain $chainfile
        %run $badlifts --map lifted.map --out good_lifted.map --log $loglifted
        
        #Creating a list of snps to include in lifted version
        snps = pd.read_csv("good_lifted.map", sep = "\t", header = None)
        snps.iloc[:,1].to_csv("snplist.txt", index = False)
        
        #Excluding snps and creating binary file
        subprocess.run(["plink", "--file", mapfile.split(".")[0], "--recode", "--out", "lifted", "--extract", "snplist.txt" ])
        subprocess.run(["plink", "--file", "--ped", "lifted.ped", "--map", "good_lifted.map", "--make-bed", "--out", outlifted])
        
        to_remove = ["lifted.ped", "lifted.map", "good_lifted.map", "snplist.txt"]
        for file in to_remove:
            os.remove(file)         

Converting MAP file to UCSC BED file...
SUCC:  map->bed succ
Lifting BED file...
SUCC:  liftBed succ
Converting lifted BED file back to MAP...
SUCC:  bed->map succ
cleaning up BED files...
Converting MAP file to UCSC BED file...
SUCC:  map->bed succ
Lifting BED file...
SUCC:  liftBed succ
Converting lifted BED file back to MAP...
SUCC:  bed->map succ
cleaning up BED files...


In [11]:
#Removing some files
for file in glob.glob("*.ped"):
    os.remove(file)
    
for file in glob.glob("*.map"):
    os.remove(file)

## QC procedure

The QC procedure runs as follows:

1. Removed founders, that is, individuals with at least one parent in the dataset, and retained only autosomal chromosomes
2. Removed SNPs with missing call rates higher than 0.1
3. Removed SNPs with minor allele frequencies below 0.05
4. Removed SNPs with hardy-weinberg equilibrium p-values less than 1e-50
5. Removed samples with missing call rates higher than 0.1
6. Removed one arbitrary individual from any pairwise comparison with a pihat >= 0.25 from an IBD estimation after LD prune

QC in all samples

In [12]:
#Move to directory and run QC procedure
os.chdir(os.path.join(pathgenos, "01_Original", "Extract_Common_ID"))
QC_procedure()

#Remove no sex individuals in CHP (plates)
#os.chdir(os.path.join(pathgenos, "01_Original", "QC"))
#for file in glob.glob("CHP*.bed"):
#    filename1 = file.split(".")[0]
#    subprocess.run(["plink", "--bfile", filename1, "--remove", filename1 + ".nosex", 
#                        "--make-bed", "--out", filename1])

Running Euro180_176ppl_317K_hg19_ATGC_CID_lifted file...
Removing SNPs with missing call rates higher than 0.1...
Removing SNPs with minor allele frequencies below 0.05...
Removing SNPs with hardy-weinberg equilibrium p-values less than 1e-50...
Removing samples with missing call rates higher than 0.1...
Removing one arbitrary individual from any pairwise comparison with a pihat higher than 0.25...
Generating final plink file...
Running ADAPT_2784ppl_567K_hg19_CID file...
Removing SNPs with missing call rates higher than 0.1...
Removing SNPs with minor allele frequencies below 0.05...
Removing SNPs with hardy-weinberg equilibrium p-values less than 1e-50...
Removing samples with missing call rates higher than 0.1...
Removing one arbitrary individual from any pairwise comparison with a pihat higher than 0.25...
Generating final plink file...
Running CV_697ppl_964K_hg19_ATGC_CID_lifted file...
Removing SNPs with missing call rates higher than 0.1...
Removing SNPs with minor allele freque

QC in _phenos subset

Moving files to 02_Clean folder, and remove some intermediary files

In [13]:
#Moving files to 02_Clean folder
os.chdir(os.path.join(pathgenos, "01_Original", "Extract_Common_ID", "QC"))
for file in glob.glob("*_rel*"):
    shutil.move(file, os.path.join(outpath, file))


Let's us look at how many SNPs and individuals ended in each dataset.

In [14]:
os.chdir(outpath)
for file in glob.glob("*_rel.log"):
    with open(file) as myfile:
        print("In file: " + file)
        for num, line in enumerate(myfile, 1):
            if "variants" in line:
                print(line, end='')
        print("Finished file... \n")

In file: ADAPT_2784ppl_567K_hg19_CID_founders_geno01_maf_hwe_mind01_rel.log
463723 variants loaded from .bim file.
463723 variants and 2289 people pass filters and QC.
Finished file... 

In file: SA_231ppl_599K_hg19_ATGC_CID_founders_geno01_maf_hwe_mind01_rel.log
458402 variants loaded from .bim file.
458402 variants and 30 people pass filters and QC.
Finished file... 

In file: Euro180_176ppl_317K_hg19_ATGC_CID_lifted_founders_geno01_maf_hwe_mind01_rel.log
295854 variants loaded from .bim file.
295854 variants and 171 people pass filters and QC.
Finished file... 

In file: CV_697ppl_964K_hg19_ATGC_CID_lifted_founders_geno01_maf_hwe_mind01_rel.log
790537 variants loaded from .bim file.
790537 variants and 154 people pass filters and QC.
Finished file... 

In file: UIUC2013_116ppl_959K_hg19_ATGC_CID_founders_geno01_maf_hwe_mind01_rel.log
774977 variants loaded from .bim file.
774977 variants and 86 people pass filters and QC.
Finished file... 

